<a href="https://colab.research.google.com/github/firoz3130/Html-samples/blob/master/sota_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import random
import json

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 10 # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 200000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path_1= "hin_train.json"
data_path_2= "ben_train.json"
data_path_3= "kan_train.json"
data_path_4= "mal_train.json"
# data_path_5= "tel_train.json"
res_path   = "res.txt"

In [ ]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines=[]

In [ ]:

temp=[]
with open(data_path_1, "r", encoding="utf-8") as f:
    line1 = f.read().split("\n")
    line1 = line1[:200000]
    for line in line1:
      data=json.loads(line)
      data=list(data.values())
      data=data[1:3]
      temp.append(data)
    line1=temp
lines += line1
len(lines)

200000

In [ ]:

# temp=[]
# with open(data_path_2, "r", encoding="utf-8") as f:
#     line2 = f.read().split("\n")
#     line2 = line2[:50000]
#     for line in line2:
#       data=json.loads(line)
#       data=list(data.values())
#       data=data[1:3]
#       temp.append(data)
#     line2=temp
# lines += line2

In [ ]:
temp=[]
with open(data_path_3, "r", encoding="utf-8") as f:
    line3 = f.read().split("\n")
    line3 = line3[:200000]
    for line in line3: 
      data=json.loads(line)
      data=list(data.values())
      data=data[1:3]
      temp.append(data)
    line3=temp
lines += line3

In [ ]:

 temp=[]
 with open(data_path_4, "r", encoding="utf-8") as f:
     line4 = f.read().split("\n")
     line4 = line4[:50000]
     for line in line4:
       data=json.loads(line)
       data=list(data.values())
       data=data[1:3]
       temp.append(data)
     line4=temp
 lines += line4

In [ ]:
random.shuffle(lines)
len(lines)

250000

In [ ]:
lines[:5]

[['सुलवेसी', 'celebes'],
 ['क्रेविरो', 'craveiro'],
 ['असकोट', 'askot'],
 ['बेन्थाम', 'bentham'],
 ['उपमहाद्वीप', 'upmahaadweep']]

In [ ]:
target_characters.add(" ")
input_characters.add(" ")

In [ ]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text,target_text= line
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 200000
Number of unique input tokens: 139
Number of unique output tokens: 29
Max sequence length for inputs: 31
Max sequence length for outputs: 34


In [ ]:
test_size = int(0.2 * num_samples)
test_size

40000

In [ ]:
test_encoder_input_data = encoder_input_data[(num_samples-test_size):]
test_decoder_target_data = decoder_target_data[(num_samples-test_size):]
test_decoder_input_data = decoder_input_data [(num_samples-test_size):]
encoder_input_data = encoder_input_data[0:(num_samples-test_size)]
decoder_target_data = decoder_target_data[0:(num_samples-test_size)]
decoder_input_data = decoder_input_data [0:(num_samples-test_size)]

In [ ]:
len(decoder_input_data ),len(test_encoder_input_data)

(160000, 40000)

In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:

model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=10,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/10
2000/2000 [==============================] - 590s 293ms/step - loss: 0.6736 - accuracy: 0.8022 - val_loss: 0.5837 - val_accuracy: 0.8228
Epoch 2/10
2000/2000 [==============================] - 590s 295ms/step - loss: 0.5379 - accuracy: 0.8353 - val_loss: 0.4965 - val_accuracy: 0.8481
Epoch 3/10
2000/2000 [==============================] - 584s 292ms/step - loss: 0.4805 - accuracy: 0.8526 - val_loss: 0.4624 - val_accuracy: 0.8575
Epoch 4/10
2000/2000 [==============================] - 582s 291ms/step - loss: 0.4199 - accuracy: 0.8699 - val_loss: 0.3811 - val_accuracy: 0.8815
Epoch 5/10
2000/2000 [==============================] - 578s 289ms/step - loss: 0.3300 - accuracy: 0.8968 - val_loss: 0.2909 - val_accuracy: 0.9097
Epoch 6/10
2000/2000 [==============================] - 582s 291ms/step - loss: 0.2607 - accuracy: 0.9185 - val_loss: 0.2414 - val_accuracy: 0.9263
Epoch 7/10
2000/2000 [==============================] - 578s 289ms/step - loss: 0.2179 - accuracy: 0.9322 - val_

In [3]:

# Define sampling models
# Restore the model and construct the encoder and decoder.


model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

OSError: ignored

In [ ]:

pred=model.predict([test_encoder_input_data,test_decoder_input_data])
actual = test_decoder_target_data

ValueError: ignored

In [ ]:
pred=pred[:, -1].astype(int)
actual = actual[:, -1].astype(int)

NameError: ignored

In [ ]:
from sklearn.metrics import f1_score,classification_report

# print(classification_report(actual, pred))

score=f1_score(actual,pred,average='micro')
score

0.0

In [ ]:
with open('res.txt', "r", encoding="utf-8") as f:
    ans = f.read().split()

encoder_result_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
for i,res_text in enumerate(ans):
  for t, char in enumerate(res_text):
      encoder_result_data[i, t, input_token_index[char]] = 1.0
  encoder_result_data[i,t + 1 :, input_token_index[" "]] = 1.0
for seq_index in range(20):
    input_seq = encoder_result_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", ans[seq_index])
    print("Decoded sentence:", decoded_sentence)

KeyError: ignored

In [ ]:

import tensorflow as tf

# Load saved Keras model
model = tf.keras.models.load_model("s2s")

# Convert to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_saved_model('s2s')
tflite_model = converter.convert()

# Save the converted model to disk
with open("model.tflite", "wb") as f:
    f.write(tflite_model)


ConverterError: ignored